# Dynamic Programming in Reinforcement Learning

In this notebook, we introduce **Dynamic Programming (DP)** : a class of algorithms used to solve **Markov Decision Processes (MDPs)** when the environment model (transitions and rewards) is **fully known**.

DP provides foundational techniques for computing **optimal policies** and **value functions**.

## 🎯 Learning Objectives

By the end of this notebook, you will be able to:
- Understand the **core idea** of Dynamic Programming in RL.
- Learn **policy evaluation**, **policy improvement**, and **policy iteration**.
- Implement **value iteration** algorithmically.
- Visualize how DP converges to an optimal policy.

## 🔹 What is Dynamic Programming?

Dynamic Programming (DP) is a method for solving complex problems by breaking them down into **simpler subproblems**.

In RL, DP helps us compute **optimal value functions** and **policies** for an MDP when we know:

- Transition probabilities `P(s'|s,a)`
- Reward function `R(s,a)`

It relies on **Bellman Equations** to update values iteratively.

## ⚖️ The Bellman Expectation Equation

For a policy `π`, the value of a state `s` is given by:

$$
V_π(s) = \sum_a π(a|s) \sum_{s'} P(s'|s,a) [R(s,a,s') + γ V_π(s')]
$$

This recursive equation expresses **how good** a state is under a given policy: it depends on expected rewards and future values.

## 🧩 Example: 3-State MDP

Let's consider a simple example:

- **States**: S1, S2, S3 (S3 is terminal)
- **Actions**: Left, Right
- **Rewards**: +1 when reaching terminal, 0 otherwise
- **Discount factor (γ)**: 0.9

In [ ]:
import numpy as np

# States: 0=S1, 1=S2, 2=S3
n_states = 3
gamma = 0.9

# Transition probabilities (deterministic for simplicity)
P = {
    0: {0: [(1.0, 1, 0)], 1: [(1.0, 2, 1)]},  # from S1, Left->S2, Right->S3
    1: {0: [(1.0, 2, 1)], 1: [(1.0, 0, 0)]},  # from S2, Left->S3, Right->S1
    2: {0: [(1.0, 2, 0)], 1: [(1.0, 2, 0)]}   # terminal S3
}

# Initialize value function
V = np.zeros(n_states)

def policy_evaluation(policy, V, theta=1e-4):
    while True:
        delta = 0
        for s in range(n_states):
            v = 0
            for a, action_prob in enumerate(policy[s]):
                for prob, next_state, reward in P[s][a]:
                    v += action_prob * prob * (reward + gamma * V[next_state])
            delta = max(delta, abs(v - V[s]))
            V[s] = v
        if delta < theta:
            break
    return V

# Define uniform random policy
policy = np.ones((n_states, 2)) / 2
V = policy_evaluation(policy, V.copy())
V

This simple **policy evaluation** estimates the expected value of each state under a uniform random policy.

## ♻️ Policy Iteration

**Policy Iteration** alternates between:

1. **Policy Evaluation** : Compute `Vπ(s)` for the current policy.
2. **Policy Improvement** : Update policy to choose the best action:

$$
π'(s) = argmax_a \sum_{s'} P(s'|s,a)[R(s,a,s') + γV(s')]
$$

In [ ]:
def one_step_lookahead(s, V):
    A = np.zeros(2)
    for a in range(2):
        for prob, next_state, reward in P[s][a]:
            A[a] += prob * (reward + gamma * V[next_state])
    return A

def policy_iteration():
    policy = np.ones((n_states, 2)) / 2
    V = np.zeros(n_states)
    stable = False

    while not stable:
        V = policy_evaluation(policy, V.copy())
        stable = True

        for s in range(n_states):
            old_action = np.argmax(policy[s])
            action_values = one_step_lookahead(s, V)
            best_action = np.argmax(action_values)
            if old_action != best_action:
                stable = False
            policy[s] = np.eye(2)[best_action]

    return policy, V

optimal_policy, optimal_values = policy_iteration()
optimal_policy, optimal_values

The algorithm converges when the policy stops changing. The resulting **optimal policy** and **value function** represent the best way to act in the environment.

## ⚡ Value Iteration

Value Iteration combines **evaluation** and **improvement** into a single update:

$$
V(s) \leftarrow \max_a \sum_{s'} P(s'|s,a)[R(s,a,s') + γV(s')]
$$

In [ ]:
def value_iteration(theta=1e-4):
    V = np.zeros(n_states)
    while True:
        delta = 0
        for s in range(n_states):
            v = V[s]
            action_values = one_step_lookahead(s, V)
            V[s] = np.max(action_values)
            delta = max(delta, abs(v - V[s]))
        if delta < theta:
            break
    policy = np.zeros((n_states, 2))
    for s in range(n_states):
        best_action = np.argmax(one_step_lookahead(s, V))
        policy[s][best_action] = 1.0
    return policy, V

v_policy, v_values = value_iteration()
v_policy, v_values

✅ **Value Iteration** directly computes the optimal value function and corresponding policy, usually faster than full policy iteration.

## 📘 Summary

- **Dynamic Programming** requires a known model of the environment.
- **Policy Evaluation** estimates how good a policy is.
- **Policy Iteration** alternates between evaluation and improvement.
- **Value Iteration** merges both into one step for efficiency.

Next, we’ll explore **Model-Free Reinforcement Learning** : where the agent learns from experience without knowing the environment model.